In [1]:
import torch
import torch.nn as nn
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
class gibbs_sampler():
    def __init__(self,latent_dim1, latent_dim2, batch_size):
        self.latent_dim1 = latent_dim1
        self.latent_dim2 = latent_dim2
        self.batch_size = batch_size
    def var_calc(self,z,g22,lambda_2):
        val   = 2*(1-torch.matmul(torch.square(z),g22)-lambda_2)
        return torch.reciprocal(val)
    def mean_calc(self,z,var,g11,lambda_1):
        beta = torch.matmul(z,g11)+lambda_1
        return var*beta
    def value_calc(self,z,g11,g22,lambda_1,lambda_2):
        var1          = self.var_calc(z,g22,lambda_2)
        mean1         = self.mean_calc(z,var1,g11,lambda_1)
        out           = mean1+torch.sqrt(var1.float())*torch.randn_like(var1)
        return out
    def gibbs_sample(self,flag,z1,z2,g11,g22,lambda_1,lambda_2,lambdap_1,lambdap_2,n_iterations):

        if flag == 1:
            z1 = torch.randn(self.batch_size,self.latent_dim1).to(device)       ## 
            z2 = torch.randn(self.batch_size,self.latent_dim2).to(device)       ## For estimating z' in  q(z'|z,x)
            
        for i in range(n_iterations):
            z1  = self.value_calc(z2,torch.transpose(g11,0,1),torch.transpose(g22,0,1),lambda_1,lambda_2) 
            z2  = self.value_calc(z1,g11,g22,lambdap_1,lambdap_2) 

        return z1,z2       